In [ ]:
# Copyright 2021 Radiotherapy AI Holdings Pty Ltd

# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at

#    http://www.apache.org/licenses/LICENSE-2.0

# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [ ]:
import pathlib
import random
import shutil
import urllib.request

import imageio
import matplotlib.pyplot as plt
import tensorflow as tf

tf.random.set_seed(42)

In [ ]:
zip_url = "https://github.com/RadiotherapyAI/unet-workshop/releases/download/mini-parotid/mini-parotid.zip"
zip_filepath = "data.zip"

data_directory = pathlib.Path("data")

if not data_directory.exists():
    urllib.request.urlretrieve(zip_url, zip_filepath)
    shutil.unpack_archive(zip_filepath, data_directory)

In [ ]:
dataset_types = [path.name for path in data_directory.glob("*") if path.is_dir()]
dataset_types

In [ ]:
def get_path_pairs(dataset_type):
    training_image_paths = list((data_directory / dataset_type).glob("*/*.image.png"))
    training_mask_paths = [
        path.parent / f"{path.name.split('.')[0]}.masks.png"
        for path in training_image_paths
    ]

    path_pairs = [
        (str(image), str(mask))
        for image, mask in zip(training_image_paths, training_mask_paths)
    ]

    return sorted(path_pairs)

In [ ]:
training_path_pairs = get_path_pairs("training")
training_path_pairs[0:5]

In [ ]:
training_dataset = tf.data.Dataset.from_tensor_slices(training_path_pairs)
training_dataset = training_dataset.shuffle(
    len(training_path_pairs), reshuffle_each_iteration=True
)

In [ ]:
validation_path_pairs = get_path_pairs("validation")
validation_dataset = tf.data.Dataset.from_tensor_slices(validation_path_pairs)
validation_dataset = validation_dataset.shuffle(
    len(validation_path_pairs), reshuffle_each_iteration=True
)

In [ ]:
@tf.function
def load(path_pair):
    image_raw = tf.io.read_file(path_pair[0])
    image = tf.io.decode_image(image_raw, channels=1, dtype=tf.uint8)

    masks_raw = tf.io.read_file(path_pair[1])
    masks = tf.io.decode_image(masks_raw, channels=3, dtype=tf.uint8)

    return image / 255, masks / 255

In [ ]:
training_dataset = training_dataset.map(load)
validation_dataset = validation_dataset.map(load)

In [ ]:
image, masks = iter(validation_dataset.take(1)).next()

In [ ]:
plt.imshow(image[:,:,0])

In [ ]:
plt.imshow(masks)

In [ ]:
colours_and_labels = [
    ("#ff7f0e", "left parotid"),
    ("#2ca02c", "right parotid"),
    ("#d62728", "external"),
]

In [ ]:
plt.figure(figsize=(12, 8))
plt.imshow(image[:,:,0], cmap="gray")

for i, (colour, label) in enumerate(colours_and_labels):
    c = plt.contour(masks[..., i], colors=[colour], levels=[0.5])
    c.collections[0].set_label(label)

plt.axis("equal")
plt.legend()

In [ ]:
def activation(x):
    x = tf.keras.layers.Activation("relu")(x)

    return x


def convolution(x, number_of_filters, kernel_size=3):
    x = tf.keras.layers.Conv2D(
        number_of_filters, kernel_size, padding="same", kernel_initializer="he_normal"
    )(x)

    return x


def conv_transpose(x, number_of_filters, kernel_size=3):
    x = tf.keras.layers.Conv2DTranspose(
        number_of_filters,
        kernel_size,
        strides=2,
        padding="same",
        kernel_initializer="he_normal",
    )(x)

    return x

In [ ]:
def encode(
    x,
    number_of_filters,
    number_of_convolutions=2,
):
    for _ in range(number_of_convolutions):
        x = convolution(x, number_of_filters)
        x = activation(x)
    skip = x

    x = tf.keras.layers.MaxPool2D()(x)
    x = activation(x)

    return x, skip


def decode(
    x,
    skip,
    number_of_filters,
    number_of_convolutions=2,
):
    x = conv_transpose(x, number_of_filters)
    x = activation(x)

    x = tf.keras.layers.concatenate([skip, x], axis=3)

    for _ in range(number_of_convolutions):
        x = convolution(x, number_of_filters)
        x = activation(x)

    return x

In [ ]:
GRID_SIZE = 64
NUM_CONTOURS = 3

In [ ]:
inputs = tf.keras.layers.Input((GRID_SIZE, GRID_SIZE, 1))

x = inputs
skips = []

for number_of_filters in [32, 64, 128]:
    x, skip = encode(x, number_of_filters)
    skips.append(skip)

skips.reverse()

for number_of_filters, skip in zip([256, 128, 64], skips):
    x = decode(x, skip, number_of_filters)

x = tf.keras.layers.Conv2D(
    NUM_CONTOURS,
    1,
    activation="sigmoid",
    padding="same",
    kernel_initializer="he_normal",
)(x)

model = tf.keras.Model(inputs=inputs, outputs=x)

In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.BinaryCrossentropy(),
    metrics=[
        tf.keras.metrics.BinaryAccuracy(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.Precision(),
    ],
)

In [ ]:
image.shape

In [ ]:
pred_masks = model.predict(image[None, ...])[0, ...]

In [ ]:
plt.figure(figsize=(12, 8))
plt.imshow(image[:,:,0], cmap="gray")

for i, (colour, label) in enumerate(colours_and_labels):
    c = plt.contour(pred_masks[..., i], colors=[colour], levels=[0.5])
    c.collections[0].set_label(label)

plt.axis("equal")
plt.legend()

In [ ]:
def plot_contours(ax, image, masks):
    ax.imshow(image[:,:,0], cmap="gray")

    for i, (colour, label) in enumerate(colours_and_labels):
        c = ax.contour(masks[..., i], colors=[colour], levels=[0.5])
        c.collections[0].set_label(label)

    ax.axis("equal")
    ax.legend()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 6), ncols=2)
plot_contours(ax[0], image, masks)
plot_contours(ax[1], image, pred_masks)

In [ ]:
class DisplayCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        fig, ax = plt.subplots(figsize=(12, 6), ncols=2)
        plot_contours(ax[0], image, masks)

        pred_masks = model.predict(image[None, ...])[0, ...]
        plot_contours(ax[1], image, pred_masks)

        plt.show()
        print("\nSample Prediction after epoch {}\n".format(epoch + 1))

In [ ]:
BATCH_SIZE = 512

batched_training_dataset = training_dataset.batch(BATCH_SIZE)
batched_training_dataset = batched_training_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
batched_validation_dataset = validation_dataset.batch(BATCH_SIZE)
batched_validation_dataset = batched_validation_dataset.prefetch(tf.data.AUTOTUNE)

In [ ]:
history = model.fit(
    batched_training_dataset,
    epochs=50,
    validation_data=batched_validation_dataset,
    callbacks=[DisplayCallback()],
)

In [ ]:
plt.semilogy(history.history['loss'], label="Training loss")
plt.semilogy(history.history['val_loss'], label="Validation loss")
plt.legend()